In [74]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [75]:
columns_names = ["user_id","movie_id","rating","timestamp"]
# taking train and test data
#please note
train = pd.read_csv("ml-100k/u5.base",sep = '\t',names = columns_names)
test = pd.read_csv("ml-100k/u5.test",sep = '\t',names = columns_names)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   user_id    80000 non-null  int64
 1   movie_id   80000 non-null  int64
 2   rating     80000 non-null  int64
 3   timestamp  80000 non-null  int64
dtypes: int64(4)
memory usage: 2.4 MB


In [76]:

# 943 users on 1682 movies
movie_matrix=np.zeros((1682,943))
#total number of rows in train dataframe
total = len(train)
#forming movie matrix (Y) from available dataset
for i in range (total):
    u_id = train["user_id"][i]-1
    m_id = train["movie_id"][i]-1
    rating= train["rating"][i]
    movie_matrix[m_id][u_id]= rating
    
Y = movie_matrix


In [77]:
Y.shape

(1682, 943)

In [78]:
# Forming mask matrix R from Y

R = np.zeros((1682,943))
for i in range(1682):
    for j in range(943):
        if(Y[i][j]==0):
            R[i][j]=0
        else :
            R[i][j]=1
#Random Matrix X 
np.random.seed(1)
X= np.random.randint(1,5,(1682,943))


In [79]:
k=500 #number of iterations
lamb =10 #threshold
Xcurr = X # X matrix for the current iteration



In [82]:
for epoch in range(1, k):
        Xprev = Xcurr #Xprev is Xmatrix from the previous iteration
        B = Xprev + Y-np.multiply(R,Xprev) # B matrix by choosing only observed values from X and subtracting it from Y and adding to X
        U,s,V = np.linalg.svd(B,full_matrices=False) #SVD of B
        
        sigma =s 
        # Now changing sigma by replacing max of 0 , s-lamb/2 at each index
        for i in range(len(s)):
            sigma[i] = max(0, s[i]- lamb/2)
        #taking diagonal of sigma
        Sigma = np.diag(sigma)
        #finding new X current from U, Sigma and V
        Xcurr = np.matmul(np.matmul(U, Sigma), V)
        
        if (np.linalg.norm((Xcurr- Xprev), 'fro')<lamb):
            continue
        else:
            break

       
        
        



In [81]:
#Now calculating MAE
# 943 users on 1682 movies
X=Xcurr
movie_matrix=np.zeros((1682,943))
#total number of rows in test dataframe
total = len(test)
error =0
#calculating MAE
#for this we take the rating from the test matrix and subtract it from the rating of X that we have claculated
# and then take its absolute
for i in range (total):
    error+=abs(test["rating"][i]-X[test["movie_id"][i]-1][test["user_id"][i]-1])

#then we divide the total error by the length of test data for MAE    
MAE = error/total
print(MAE)

# MAE = 0.7722060183441429

0.7722060183441429
